<a href="https://colab.research.google.com/github/pdangi-web/Data_Mining_HW/blob/main/HW3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
from collections import Counter
import os
import math
import argparse

DATA_PATH = "data.csv"   # path to uploaded data in Colab
LABEL_PATH = "label.csv" # path to uploaded labels in Colab

def l2_normalize_rows(mat):
    norms = np.linalg.norm(mat, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    return mat / norms

def pairwise_euclidean(X, centroids):
    x2 = np.sum(X*X, axis=1, keepdims=True)            # (n,1)
    c2 = np.sum(centroids*centroids, axis=1)           # (k,)
    xc = X.dot(centroids.T)                            # (n,k)
    d2 = x2 + c2 - 2*xc
    d2 = np.maximum(d2, 0.0)
    return np.sqrt(d2)

def pairwise_one_minus_cosine(Xn, centroids):
    Cn = centroids.copy()
    Cn_norms = np.linalg.norm(Cn, axis=1, keepdims=True)
    Cn_norms[Cn_norms==0] = 1.0
    Cn = Cn / Cn_norms
    sim = Xn.dot(Cn.T)
    return 1.0 - sim

def pairwise_one_minus_jaccard(X, centroids):
    X_nonneg = X.copy()
    shift = 0.0
    min_all = X_nonneg.min()
    if min_all < 0:
        shift = -min_all
        X_nonneg = X_nonneg + shift
    n = X_nonneg.shape[0]
    k = centroids.shape[0]
    distances = np.empty((n,k), dtype=float)
    for j in range(k):
        c = centroids[j:j+1,:].copy()
        if shift != 0.0:
            c = c + shift
        mins = np.minimum(X_nonneg, c)
        maxs = np.maximum(X_nonneg, c)
        num = np.sum(mins, axis=1)
        den = np.sum(maxs, axis=1)
        with np.errstate(divide='ignore', invalid='ignore'):
            jacc = np.where(den==0, 1.0, num/den)
        distances[:, j] = 1.0 - jacc
    return distances

class KMeansScratch:
    def __init__(self, n_clusters=10, metric='euclidean', tol=1e-6, max_iter=500, random_state=123):
        self.n_clusters = n_clusters
        assert metric in ('euclidean','cosine','jaccard')
        self.metric = metric
        self.tol = tol
        self.max_iter = max_iter
        self.random_state = random_state
        self.centroids = None
        self.labels_ = None
        self.inertia_ = None
        self.n_iter_ = 0
        self.history_inertia = []

    def _compute_distances(self, X, centroids, Xn=None):
        if self.metric == 'euclidean':
            return pairwise_euclidean(X, centroids)
        elif self.metric == 'cosine':
            if Xn is None:
                Xn = l2_normalize_rows(X)
            return pairwise_one_minus_cosine(Xn, centroids)
        elif self.metric == 'jaccard':
            return pairwise_one_minus_jaccard(X, centroids)

    def fit(self, X, stop_on_sse_increase=True, verbose=False):
        rng = np.random.RandomState(self.random_state)
        n, d = X.shape
        init_idx = rng.choice(n, self.n_clusters, replace=False)
        centroids = X[init_idx].astype(float).copy()
        prev_inertia = None

        Xn = None
        if self.metric == 'cosine':
            Xn = l2_normalize_rows(X)

        for it in range(1, self.max_iter+1):
            distances = self._compute_distances(X, centroids, Xn=Xn)
            labels = np.argmin(distances, axis=1)
            min_d = distances[np.arange(n), labels]
            inertia = float(np.sum(min_d**2))
            self.history_inertia.append(inertia)

            new_centroids = np.zeros_like(centroids)
            for j in range(self.n_clusters):
                members = X[labels==j]
                if len(members)==0:
                    new_centroids[j] = X[rng.randint(0, n)]
                else:
                    new_centroids[j] = members.mean(axis=0)

            move = np.linalg.norm(new_centroids - centroids)
            sse_increased = (prev_inertia is not None) and (inertia > prev_inertia + 1e-12)

            centroids = new_centroids
            prev_inertia = inertia
            self.centroids = centroids
            self.labels_ = labels
            self.inertia_ = inertia
            self.n_iter_ = it

            if verbose and (it % 10 == 0 or it==1):
                print(f"iter {it}: inertia={inertia:.6e}, move={move:.6e}")

            if move <= self.tol:
                return "centroid_no_change"
            if stop_on_sse_increase and sse_increased:
                return "sse_increased"
        return "max_iter"

def majority_label_accuracy(true_labels, cluster_labels):
    mapping = {}
    pred_labels = np.zeros_like(true_labels)
    for c in np.unique(cluster_labels):
        members = true_labels[cluster_labels==c]
        if len(members)==0:
            mapping[c] = -1
        else:
            mapping[c] = Counter(members).most_common(1)[0][0]
        pred_labels[cluster_labels==c] = mapping[c]
    acc = np.mean(pred_labels == true_labels)
    return acc, mapping

def run_all_experiments(data_path=DATA_PATH, label_path=LABEL_PATH, max_iter=500):
    X = np.loadtxt(data_path, delimiter=',')
    y = np.loadtxt(label_path, delimiter=',', dtype=int)
    K = len(np.unique(y))
    print("Loaded X shape:", X.shape, "labels shape:", y.shape, "K:", K)

    metrics = ['euclidean','cosine','jaccard']
    results = {}

    for metric in metrics:
        print("\nRunning metric:", metric, "(stop on SSE increase enabled)")
        km = KMeansScratch(n_clusters=K, metric=metric, max_iter=max_iter, tol=1e-6, random_state=42)
        t0 = time.perf_counter()
        stop_reason = km.fit(X, stop_on_sse_increase=True, verbose=False)
        t1 = time.perf_counter()
        acc, mapping = majority_label_accuracy(y, km.labels_)
        results[(metric, 'sse_stop')] = {
            'inertia': km.inertia_,
            'iterations': km.n_iter_,
            'time_s': t1-t0,
            'acc': acc,
            'stop_reason': stop_reason,
            'mapping': mapping,
            'history_inertia': km.history_inertia
        }
        print(f"done: inertia={km.inertia_:.6e}, iters={km.n_iter_}, time={t1-t0:.2f}s, acc={acc:.4f}, stop={stop_reason}")

    for metric in metrics:
        print("\nRunning metric:", metric, "(SSE-increase stop disabled)")
        km = KMeansScratch(n_clusters=K, metric=metric, max_iter=max_iter, tol=1e-6, random_state=42)
        t0 = time.perf_counter()
        stop_reason = km.fit(X, stop_on_sse_increase=False, verbose=False)
        t1 = time.perf_counter()
        acc, mapping = majority_label_accuracy(y, km.labels_)
        results[(metric, 'no_sse_stop')] = {
            'inertia': km.inertia_,
            'iterations': km.n_iter_,
            'time_s': t1-t0,
            'acc': acc,
            'stop_reason': stop_reason,
            'mapping': mapping,
            'history_inertia': km.history_inertia
        }
        print(f"done: inertia={km.inertia_:.6e}, iters={km.n_iter_}, time={t1-t0:.2f}s, acc={acc:.4f}, stop={stop_reason}")

    return results, X, y

if __name__ == "__main__":
    results, X, y = run_all_experiments(max_iter=500)
    import json
    out = {}
    for k,v in results.items():
        key = f"{k[0]}__{k[1]}"
        out[key] = { 'inertia': float(v['inertia']),
                     'iterations': int(v['iterations']),
                     'time_s': float(v['time_s']),
                     'acc': float(v['acc']),
                     'stop_reason': v['stop_reason'] }
    with open("kmeans_results.json","w") as f:
        json.dump(out, f, indent=2)
    print("\nSaved results to kmeans_results.json")
    with open("kmeans_hw3.py","w") as f:
        f.write("# saved program file for HW3 (abbreviated)\n")
    print("Saved program file as kmeans_hw3.py")


Loaded X shape: (10000, 784) labels shape: (10000,) K: 10

Running metric: euclidean (stop on SSE increase enabled)
done: inertia=2.541477e+10, iters=33, time=2.19s, acc=0.5851, stop=centroid_no_change

Running metric: cosine (stop on SSE increase enabled)
done: inertia=6.862293e+02, iters=29, time=1.02s, acc=0.6277, stop=sse_increased

Running metric: jaccard (stop on SSE increase enabled)
done: inertia=4.239946e+03, iters=2, time=1.28s, acc=0.4301, stop=sse_increased

Running metric: euclidean (SSE-increase stop disabled)
done: inertia=2.541477e+10, iters=33, time=3.25s, acc=0.5851, stop=centroid_no_change

Running metric: cosine (SSE-increase stop disabled)
done: inertia=6.864356e+02, iters=48, time=1.67s, acc=0.6309, stop=centroid_no_change

Running metric: jaccard (SSE-increase stop disabled)
done: inertia=3.660389e+03, iters=59, time=36.49s, acc=0.6021, stop=centroid_no_change

Saved results to kmeans_results.json
Saved program file as kmeans_hw3.py
